In [1]:
from ebrec.models.newsrec.model_config import hparams_lstur
from ebrec.models.newsrec.lstur import LSTURModel
import numpy as np

config = hparams_lstur

# Define the number of samples in your batch
BATCH_SIZE = 300
HISTORY_SIZE = config.history_size
TITLE_SIZE = config.title_size
NPRATIO = 4
word_embeddings = np.random.rand(1000, 100)

# Define the shapes of the input data
his_input_title_shape = (HISTORY_SIZE, TITLE_SIZE)
pred_input_title_shape = (NPRATIO + 1, TITLE_SIZE)
vocab_size = word_embeddings.shape[0]
n_users = config.n_users
label_shape = (NPRATIO + 1,)
user_indexes_shape = (1,)

model = LSTURModel(hparams=config, word2vec_embedding=word_embeddings)
model.model.summary()

# Generate some random input data for input_1 with values between 0 and 1
his_input_title = np.random.randint(0, vocab_size, (BATCH_SIZE, *his_input_title_shape))
# Generate some random input data for input_2 with values between 0 and 1
pred_input_title = np.random.randint(0, vocab_size, (BATCH_SIZE, *pred_input_title_shape))
# Input data for user_indexes
user_indexes = np.random.randint(0, n_users, size=(BATCH_SIZE, *user_indexes_shape))

# Generate some random label data with values between 0 and 1
label_data = np.zeros((BATCH_SIZE, *label_shape), dtype=int)
for row in label_data:
    row[np.random.choice(label_shape[0])] = 1

# Print the shapes of the input data to verify they match the model's input layers
print(his_input_title.shape)
print(pred_input_title.shape)
print(user_indexes.shape)
print(label_data.shape)

# Make input for model:
input = (user_indexes, his_input_title, pred_input_title)

# fit/predict:
model.model.fit(input, label_data)
model.model.predict(input)


2024-05-11 00:21:34.176197: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-05-11 00:21:34.176225: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-11 00:21:34.176234: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-11 00:21:34.176270: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-11 00:21:34.176284: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='att_layer2/Sum_1:0', description="created by layer 'att_layer2'")


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, 30)]           0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 50, 30)]             0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 time_distributed_1 (TimeDi  (None, None, 400)            300800    ['input_2[0][0]']             
 stributed)                                                                                   

2024-05-11 00:21:36.817586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-05-11 00:21:36.945177: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp_14.


10/10 [==============================] - 11s 461ms/step - loss: 1.6133


W0000 00:00:1715379707.016849       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } attr { key: "_has_manual_control_dependencies" value { b: true } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


10/10 [==============================] - 1s 49ms/step


array([[0.20042472, 0.22490296, 0.20391975, 0.1897768 , 0.18097578],
       [0.19249606, 0.21868879, 0.2050681 , 0.17383885, 0.20990819],
       [0.22947909, 0.21508436, 0.18182603, 0.17216888, 0.20144172],
       ...,
       [0.1854658 , 0.18727688, 0.22403893, 0.20266993, 0.20054844],
       [0.20902286, 0.21204741, 0.19030018, 0.2216475 , 0.16698207],
       [0.19700955, 0.1825246 , 0.20179047, 0.20683563, 0.21183978]],
      dtype=float32)

<br><br><br><br>

In [7]:
from pathlib import Path
import tensorflow as tf
import polars as pl
import os

from ebrec.utils._constants import *
from ebrec.utils._behaviors import create_binary_labels_column, sampling_strategy_wu2019, add_known_user_column, add_prediction_scores, truncate_history
from ebrec.utils._articles import convert_text2encoding_with_transformers, create_article_id_to_value_mapping
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._nlp import get_transformers_word_embeddings

from transformers import AutoTokenizer, AutoModel
from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel


"""
load data
"""

data_base = Path(os.getcwd()).parent / "data-merged" / "merged"
# train_val_base = data_base / "1-ebnerd_demo_(20MB)"
train_val_base = data_base / "2-ebnerd_small_(80MB)"
# train_val_base = data_base / "3-ebnerd_large_(3.0GB)"
test_base = data_base / "5-ebnerd_testset_(1.5GB)"
assert train_val_base.exists() and test_base.exists()

train_behaviors = pl.scan_parquet(train_val_base / "train" / "behaviors.parquet")
train_history = pl.scan_parquet(train_val_base / "train" / "history.parquet")

val_behavior = pl.scan_parquet(train_val_base / "validation" / "behaviors.parquet")
val_history = pl.scan_parquet(train_val_base / "validation" / "history.parquet")

test_behavior = pl.scan_parquet(test_base / "test" / "behaviors.parquet")
test_history = pl.scan_parquet(test_base / "test" / "history.parquet")

train_articles: pl.LazyFrame = pl.scan_parquet(train_val_base / "articles.parquet")
val_articles: pl.LazyFrame = train_articles
test_articles: pl.LazyFrame = pl.scan_parquet(test_base / "articles.parquet")

articles_word2vec: pl.LazyFrame = pl.scan_parquet(data_base / "7-Ekstra-Bladet-word2vec_(133MB)" / "document_vector.parquet")
articles_image_embeddings: pl.LazyFrame = pl.scan_parquet(data_base / "8-Ekstra_Bladet_image_embeddings_(372MB)" / "image_embeddings.parquet")
articles_contrastive_vector: pl.LazyFrame = pl.scan_parquet(data_base / "9-Ekstra-Bladet-contrastive_vector_(341MB)" / "contrastive_vector.parquet")
articles_bert_base_multilingual_cased: pl.LazyFrame = pl.scan_parquet(data_base / "10-google-bert-base-multilingual-cased_(344MB)" / "bert_base_multilingual_cased.parquet")
articles_xlm_roberta_base: pl.LazyFrame = pl.scan_parquet(data_base / "11-FacebookAI-xlm-roberta-base_(341MB)" / "xlm_roberta_base.parquet")


"""
preprocessing: truncate user history, select subset of columns, join behavior and history, sample based on Wu2019, add binary labels column
"""

def ebnerd_from_path(history: pl.LazyFrame, behaviors: pl.LazyFrame, history_size: int = 30) -> pl.DataFrame:
    df_history = history \
            .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL) \
            .pipe(truncate_history, column=DEFAULT_HISTORY_ARTICLE_ID_COL, history_size=history_size, padding_value=0)
    df_behaviors = behaviors.collect() \
            .pipe(slice_join_dataframes, df2=df_history.collect(), on=DEFAULT_USER_COL, how="left")
    return df_behaviors

COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
]
HISTORY_SIZE = 30
N_SAMPLES = 100
df_train = ebnerd_from_path(history=train_history, behaviors=train_behaviors, history_size=HISTORY_SIZE) \
            .select(COLUMNS) \
            .pipe(sampling_strategy_wu2019, npratio=4, shuffle=True, with_replacement=True, seed=123) \
            .pipe(create_binary_labels_column) \
            .sample(n=N_SAMPLES)
df_validation = ebnerd_from_path(history=val_history, behaviors=val_behavior, history_size=HISTORY_SIZE) \
            .select(COLUMNS) \
            .pipe(create_binary_labels_column) \
            .sample(n=N_SAMPLES)
df_test = ebnerd_from_path(history=test_history, behaviors=val_behavior, history_size=HISTORY_SIZE) \
            .with_columns(pl.Series(DEFAULT_CLICKED_ARTICLES_COL, [[]])) \
            .select(COLUMNS) \
            .pipe(create_binary_labels_column) \
            .sample(n=N_SAMPLES)


"""
use huggingface transformers to convert article text to tokens, tokens to embeddings
"""

df_articles = train_articles.collect()
TRANSFORMER_MODEL_NAME = "bert-base-multilingual-cased"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

word2vec_embedding = get_transformers_word_embeddings(transformer_model)
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH)
article_mapping = create_article_id_to_value_mapping(df=df_articles, value_col=token_col_title)


"""
batch data
"""

train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=64,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)
test_dataloader = NRMSDataLoader(
    behaviors=df_test,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)


"""
train model
"""
MODEL_NAME = "NRMS"
LOG_DIR = f"../runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"../runs/data/state_dict/{MODEL_NAME}/weights"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
modelcheckpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1)

hparams_nrms.history_size = HISTORY_SIZE
model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=1,
    callbacks=[tensorboard_callback, early_stopping_callback, modelcheckpoint_callback],
)
model.model.load_weights(filepath=MODEL_WEIGHTS)


2024-05-11 00:18:36.960394: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-05-11 00:18:36.960615: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-11 00:18:36.960624: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-11 00:18:36.960668: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-11 00:18:36.960981: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-05-11 00:18:39.318974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-05-11 00:18:39.429383:

2/2 [==============================] - ETA: 0s - loss: 1.6096
Epoch 1: val_loss improved from inf to 0.00000, saving model to ../runs/data/state_dict/NRMS/weights
2/2 [==============================] - 65s 57s/step - loss: 1.6096 - val_loss: 0.0000e+00


# evaluate performance

In [8]:
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore

pred_validation = model.scorer.predict(val_dataloader)

df_validation = add_prediction_scores(df_validation, pred_validation.tolist()) \
    .pipe(add_known_user_column, known_users=df_train[DEFAULT_USER_COL])

metrics = MetricEvaluator(
    labels=df_validation["labels"].to_list(),
    predictions=df_validation["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

4/4 [==============================] - 3s 575ms/step


<MetricEvaluator class>: 
 {
    "auc": 0.5068132911024271,
    "mrr": 0.301223308572205,
    "ndcg@5": 0.3368249540891147,
    "ndcg@10": 0.43714083395568126
}

generate submission file

In [9]:
# pred_test = model.scorer.predict(test_dataloader) <--- breaks because of size mismatch
pred_test = model.scorer.predict(val_dataloader)
print(pred_test)

# store
submission_path = Path(os.getcwd()).parent / "submissions" / "ebnerd_nrms.txt"
with open(submission_path, "w") as f:
    for idx, row in enumerate(pred_test):
        f.write(f"{idx} {row}\n")
    f.close()

4/4 [==============================] - 2s 665ms/step
[[0.49938118]
 [0.4980004 ]
 [0.49865976]
 ...
 [0.50148994]
 [0.4993194 ]
 [0.49926835]]
